# 4.0 Results analysis

In [1]:
import numpy as np
from cyclops.experiment import Experiment
from cyclops.fields import ScalarField
from cyclops.object_reader import PickleManager
from cyclops.optimisers import NSGA2Optimiser
from cyclops.plotting import PlotManager
from cyclops.regressors import RBFModel
from cyclops.sensor_suite import SensorSuite
from cyclops.sensors import Thermocouple

# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = PlotManager()
true_temp_field = pickle_manager.read_file('results/temp_line_field.pickle')
grid = pickle_manager.read_file('results/temp_line_points.pickle')
res = pickle_manager.read_file('results/temp_1D_res_RBF5.pickle')
field_bounds = true_temp_field.get_bounds()
sensor_bounds = field_bounds+np.array([[1], [-1]])*0.002

# Setup the experiment
optimiser = NSGA2Optimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)

# Setup the sensor suite
temps = pickle_manager.read_file('../sensors/k-type-T.pickle')
voltages = pickle_manager.read_file('../sensors/k-type-V.pickle')
sensor = Thermocouple(temps, voltages, 1)
sensors = np.array([sensor]*5)

sensor_suite = SensorSuite(
    ScalarField(RBFModel, field_bounds), 
    sensors
)
experiment.plan_moo(
    sensor_suite,
    sensor_bounds,
    repetitions=10,
    loss_limit=1000,
    num_cores=4
)

FileNotFoundError: [Errno 2] No such file or directory: 'results/temp_1D_res_RBF5.pickle'

This code is for when you get the results from the supercomputer then copy them into the results folder and want to analyse them. Remember that you must setup the experiment and sensor layout with the same parameters.

In [ ]:
graph_manager.build_pareto(res.F)
graph_manager.draw()

graph_manager.build_pareto(res.F)

display_str = input('Enter setup to display [Q to quit]: ')
while display_str.isnumeric():
    proposed_layout, true_temps, model_temps, sensor_values = experiment.get_SOO_plotting_arrays(res.X[int(display_str)])
    print('\nLoss:', experiment.calc_moo_loss(res.X[int(display_str)]))
    graph_manager.build_1D_compare(
        grid,
        proposed_layout,
        sensor_values,
        true_temps,
        model_temps
    )
    graph_manager.draw()
    display_str = input('Enter setup to display [Q to quit]: ')

NameError: name 'res' is not defined